# TmVO4 neutrons data analysis
Fit neutrons diffraction peaks measured on TmVO4 at SNS on 2019-02-14 in order to extract the orthorhombic distortion as a function of magnetic field

## Import modules

In [114]:
from mpl_toolkits.mplot3d import Axes3D# for 3D plotting
import matplotlib.tri as mtri# for triangulation of unevenly separated data, like our magnetic field data

import copy as cp, numpy as np, pandas as pd, pickle, os, re
from datetime import date

import importlib, sys 
import matplotlib
from matplotlib import cm, pyplot as plt, rcsetup, rc, rcParams# import matplotlib.pyplot as plt
# cm stands for colormap
from matplotlib.ticker import LogLocator, LinearLocator, FormatStrFormatter
from scipy.interpolate import griddata
from scipy.special import erfc, exp1

from lmfit import minimize, Model, Parameters, report_fit, fit_report
from lmfit.models import LinearModel
# import pytest

## Import data

### Coarse data

In [2]:
tempPath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14'
os.chdir(tempPath)
(_, _, filenames) = next(os.walk(tempPath))# the walk() function lists the content of the directory that it is given as argument,
# and of its subdirectories; the next() function returns the next output of the walk() function;
# when used only once, it returns only the first output which is the content of the parent directory, 
# listed as a tuple of the form (dirpath, dirnames, filenames)
filenames = [filename for filename in filenames if 'p6K_' in filename]# keep only files that match the string pattern of datafilestempList = [None]*len(filenames)# Pre-allocate temporary list to store the data that will then be converted into a Pandas DataFrame

tempList = [None]*len(filenames)
for idx, filename in enumerate(filenames):# For each data file
    H = float(re.split('p6K_(\w*)T\w*.txt',filenames[idx])[1].replace('p','.'))# Extract value of magnetic field from filename
    tempDF = pd.read_csv(filenames[idx],names=["hh0","I","dI"],skiprows=2,delimiter=',')# import data as a Pandas DataFrame
    tempList[idx] = {'filename': filenames[idx],# Store into a dictionary, which is itself an element of tempList: the filename,
                   'H (T)': H,# value of magnetic field,
                   'T (K)': 0.6,# temperature,
                   'spectra': tempDF# and data stored as Pandas dataframe
                   }
coarseData = pd.DataFrame(tempList)# Convert the list of dictionaries into a Pandas DataFrame
del idx, H, tempPath, filename, filenames, tempList, tempDF# delete temporary variables after use 
coarseData# Show the resulting DataFrame

,filename,H (T),T (K),spectra
0,p6K_1T.txt,1.00,0.6,hh0 I dI 0 -11.99...
1,p6K_1T_new.txt,1.00,0.6,hh0 I dI 0 -11.99...
2,p6K_p05T.txt,0.05,0.6,hh0 I dI 0 -11.9975...
3,p6K_p4T.txt,0.40,0.6,hh0 I dI 0 -11.997...
4,p6K_p5T.txt,0.50,0.6,hh0 I dI 0 -11.99...
5,p6K_p6T.txt,0.60,0.6,hh0 I dI 0 -11.99...
6,p6K_p77T.txt,0.77,0.6,hh0 I dI 0 -11.99...
7,p6K_p7T.txt,0.70,0.6,hh0 I dI 0 -11.99...
8,p6K_p83T.txt,0.83,0.6,hh0 I dI 0 -11.99...
9,p6K_p97T.txt,0.97,0.6,hh0 I dI 0 -11.99...


In [3]:
coarseData.spectra[3].head()# Check the content of individual datasets after importation

,hh0,I,dI
0,-11.9975,142028.0,52309.7
1,-11.9925,220372.0,62925.4
2,-11.9875,220002.0,62818.9
3,-11.9825,518428.0,92939.8
4,-11.9775,817317.0,127501.0


### Fine data

In [4]:
tempPath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14\p6K\linecut_f'
os.chdir(tempPath)
fieldInfo = pd.read_csv('field_info.txt',header=None,names=['File #','T (K)','H (T)','Proton charge'],delimiter=' ')
fieldInfo['File #'] = fieldInfo['File #'].astype('int')# replace file number type from float to int
fieldInfo.head()
# fieldInfo['File #'].dtype# check that the change of datatype is effective

,File #,T (K),H (T),Proton charge
0,88631,0.605973,0.000000,1.312139
1,88632,0.613334,0.049998,0.800119
2,88633,0.605727,0.099996,0.083534
3,88634,0.625493,0.099996,0.800739
4,88635,0.618220,0.150005,0.801294


In [5]:
tempList = [None]*len(fieldInfo)# Preallocate list to store the data that will then be converted into a Pandas DataFrame
for idx in range(len(fieldInfo)):# For each data file
    filename = ''.join(['HH0_',str(fieldInfo['File #'][idx]),'.txt'])
#     print(filename)
    tempDF = pd.read_csv(filename,names=["hh0","I","dI"],skiprows=2,delimiter=',')# Import data as a Pandas DataFrame
    tempList[idx] = {'filename': filename,# Store into a dictionary, which is itself an element of dfList: the filename,
                     'T (K)': fieldInfo['T (K)'][idx],# temperature,
                     'H (T)': fieldInfo['H (T)'][idx],# value of magnetic field,
                     'Proton charge': fieldInfo['Proton charge'][idx],# proton charge,
                     'spectra': tempDF# and data,
                     }
linecut_f_raw = pd.DataFrame(tempList)# Convert the list of dictionaries into a Pandas DataFrame
linecut_f_raw.head()# Show the resulting DataFrame

,filename,T (K),H (T),Proton charge,spectra
0,HH0_88631.txt,0.605973,0.000000,1.312139,hh0 I dI 0 -12.99880 0.0 ...
1,HH0_88632.txt,0.613334,0.049998,0.800119,hh0 I dI 0 -12.99880 0.0 ...
2,HH0_88633.txt,0.605727,0.099996,0.083534,hh0 I dI 0 -12.99880 0.0 ...
3,HH0_88634.txt,0.625493,0.099996,0.800739,hh0 I dI 0 -12.99880 0.0 ...
4,HH0_88635.txt,0.618220,0.150005,0.801294,hh0 I dI 0 -12.99880 0.0 ...


In [6]:
del idx, tempPath, filename, tempList, tempDF# delete temporary variables after use 

In [7]:
linecut_f_raw['spectra'][3].loc[180:185]#.head()# Check the content of individual datasets after importation

,hh0,I,dI
180,-12.5487,0.0,0.0
181,-12.5462,0.0,0.0
182,-12.5437,0.0,0.0
183,-12.5412,0.0,0.0
184,-12.5388,0.0,0.0
185,-12.5363,0.0,0.0


## Basic data processing
Rescale data and remove "bad" data

### Check consistency of dataframes

#### <a name="hh0_consistency"></a>Check that all hh0 data are the same within each dataframe
i.e. that all hh0 of coarseData are the same and that all hh0 data of linecut_f_raw are the same

In [8]:
nData_raw = [coarseData,linecut_f_raw]
for data_idx in range(len(nData_raw)):# for each dataset
    for _, row in nData_raw[data_idx].iterrows():# loop over all rows
        if not np.array_equal(row.spectra.hh0,nData_raw[data_idx].spectra[0].hh0):
        # and compare the array of hh0 of that row with that of the first row
            print(row)# print the row if the two arrays are *not* equal
            # should output nothing, which means that all arrays of hh0 are the same *within a dataset*

#### Then check that hh0 data of coarseData differ from that of linecut_f_raw
We do not need to loop over all rows since we have shown in the previous cell that all rows are the same within a dataset

In [9]:
if not np.array_equal(coarseData.spectra[0].hh0,linecut_f_raw.spectra[0].hh0):
# compare the hh0 arrays of the first row of both datasets
    print("The arrarys of hh0 are not the same in both datasets")

The arrarys of hh0 are not the same in both datasets


### Which datasets to analyze and how? 

#### Update 2020-04-06
In fact, ignore the coarse dataset, as it will require a lot of efforts for a minimal result.
More interesting would be to analyze the linecut_f dataset at each (hh0) peak position, for h = 6, 8, 10.

##### Update 2020-04-03
treat both datasets independently in terms of the plotting and fitting

##### Outdated ideas
involving treating both datasets together, which will make things more complicated, and therefore increases the risks of making errors, in addition to increasing the time required for the analysis:
* interpolate spectra of coarseData so that its hh0 array is the same as that of linecut_f_raw
* rescale the data, if there is a physical way to do it, otherwise simply treat both datasets separately

### Truely "deep" copy of linecut_f
such that linecut_f_raw will *not* be modified if lcf_copy is modified

The best way to truely, i.e. recursively, deep copy a python object is to `pickle.dump` and `pickle.load` it. 
That is because `cPickle` is the fastest, as shown [here](https://stackoverflow.com/questions/1410615/copy-deepcopy-vs-pickle),  *and* **in Python 3**, `cPickle` is the default behavior of `pickle`, as explained [here](https://askubuntu.com/a/804618).
See also hacks.ipynb, and [here](https://stackoverflow.com/questions/52708341/make-a-truly-deep-copy-of-a-pandas-series). 

In [10]:
lcf_copy = pickle.loads(pickle.dumps(linecut_f_raw))# fastest python hack to create a truely deep copy
# lcf stands for linecut_f, which is the name of the file from which the data was imported
# idx = 5
lcf_copy.head()#.loc[idx:idx+10]

,filename,T (K),H (T),Proton charge,spectra
0,HH0_88631.txt,0.605973,0.000000,1.312139,hh0 I dI 0 -12.99880 0.0 ...
1,HH0_88632.txt,0.613334,0.049998,0.800119,hh0 I dI 0 -12.99880 0.0 ...
2,HH0_88633.txt,0.605727,0.099996,0.083534,hh0 I dI 0 -12.99880 0.0 ...
3,HH0_88634.txt,0.625493,0.099996,0.800739,hh0 I dI 0 -12.99880 0.0 ...
4,HH0_88635.txt,0.618220,0.150005,0.801294,hh0 I dI 0 -12.99880 0.0 ...


### Data rescaling
This used to be done manually, after noticing that the intensity of the data at 0T was higher than that of the rest of the data. A factor of 0.635 was then used to rescale this spectrum to the level of other data. 
However, after getting the up-to-date data, it appears that the scaling factor is merely the proton charge, which is a proxy for the counting time of neutrons. With this information, it turns out that the ratio of Proton charges of the spectrum at 0T and that at 0.05T is 1.3 to 0.8. The rescaling factor of the former is thus 0.8/1.3=0.615. Hence the empirical value of 0.635 was a pretty good guess!

In [11]:
##### Rescale data according to their Proton charge
for idx, row in lcf_copy.iterrows():
    row.spectra['Inorm'] = row.spectra.I/row['Proton charge']
    row.spectra['dInorm'] = row.spectra.dI/row['Proton charge']
#     print(idx, row['spectra']['I'])
idx = 1200# index that allows to look at data close to the (10 10 0) peak
lcf_copy.spectra[0].loc[idx:idx+5]# check one of the resulting dataframes

,hh0,I,dI,Inorm,dInorm
1200,-9.99875,1261600.0,94684.7,9.614833e+05,72160.557743
1201,-9.99625,1230390.0,98819.2,9.376977e+05,75311.519049
1202,-9.99375,1448420.0,109920.0,1.103862e+06,83771.596753
1203,-9.99125,2141070.0,124810.0,1.631740e+06,95119.477718
1204,-9.98875,2560080.0,141466.0,1.951073e+06,107813.252422
1205,-9.98625,4044660.0,169302.0,3.082493e+06,129027.464278


### Find bad data, if any
#### Identify datasets with zero intensity and create a new "clean" dataset without bad data

In [12]:
lcf_clean = lcf_copy.copy()
# this is a deepcopy according to Pandas, which is only deep at the lowest order, i.e. not recursively
for idx, row in lcf_copy.iterrows():# loop over all spectra
    if not np.any(row.spectra.Inorm>0):# if any spectrum has a constant zero intensity
        lcf_clean = lcf_copy.drop(idx)
        print(idx)# output the row index of lcf_copy containing empty data
# lcf_copy.spectra[0].Inorm

84


### Change default plotting parameters
see https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.figure.html

In [13]:
%matplotlib qt
# plot figures in external window

In [14]:
rcParams["figure.figsize"] = np.multiply([6.4, 4.8],0.5)# default is [6.4, 4.8]

### Batch plot the rest of the spectra to identify other potential bad data

##### Plot spectra by groups of nSpec
to see if any spectrum differs from the others.
This method may fail if there are more than nSpec bad consecutive spectra. Hence nSpec should not be too small (5 or more should be good)

In [15]:
nSpec = 7# number of curves in each plot
nfig = len(lcf_clean)//nSpec+1
fig = [None]*(nfig)
lgd = [None]*(nfig)
for fidx in range(1):#range(nfig):
    fig[fidx] = plt.figure()
    ax = fig[fidx].add_subplot(1,1,1)
    for pidx in range(nSpec*fidx,nSpec*(fidx+1)):
        try:
            plt.plot(lcf_clean.spectra[pidx].hh0,lcf_clean.spectra[pidx].Inorm,\
                     label=f"{pidx}, {lcf_clean['H (T)'][pidx]:.2f}")
        except KeyError:# if the index of data to be plotted does not exist
            continue# ignore and carry on to the next one
    plt.xlim(-10.25,-5.75)
    lgd[fidx] = ax.legend(title='Index, H (T)')
    lgd[fidx].set_draggable(True)
    plt.show()# ensures that all windows come to the foreground

##### Notes --- 2020-04-07
* After plotting all spectra together, it appears that only spectra #2 and #85 are "bad": the former is very noisy (not too surprising given that its Proton charge is 1/10 of the other data) and the latter has a lower intensity than the other spectra (for an unknown reason).
* Both those bad spectra were identified during the measurement, such that another (good) spectrum was measured at both values of magnetic field at which those bad spectra had been measured. Concretely, spectrum #2 was measured at 0.1T and #85 at 0.865T but are bad. Spectra #3 and #104 were also measured at 0.1T and 0.865T, respectively, and are good. Hence, the two bad spectra should just be discarded, there is no reason to try and process them in order to try and make them good: first because there is no way of making good data out of bad data (bad data is just bad data), so it would be a waste of time, and potentially a lot of time, but it would also be complicated, and because the result cannot be good, it can only influence the result of the subsequent fits in a bad way, thus inducing distrust on the results obtained over the whole dataset instead of just two spectra. Bottom line: discard spectra #2 and #85.

### Remove bad data 
as identified in batch plotting

In [16]:
delRowIdx = [2,85]# index of data to remove
lcf_clean.drop(delRowIdx,inplace=True)

### Final clean dataset
#### Sorted by value of magnetic field

In [17]:
nData = lcf_clean.sort_values(by=['H (T)'],ignore_index=True)
# lcf_clean.head()# nData stands for "neutrons Data"

#### Plot a couple of ENS spectra (ignore after done once)
to check consistency between spectra before fitting

### Plot entire dataset in 3D
as spectrum normalized intensity 'Inorm' vs position in reciprocal space 'hh0' and magnetic field 'H (T)', to check that the field dependence of the data is consistent.
#### Ignore after done once
Change cells to Raw type in order to avoid running inadvertantly

#### First plot individual spectra in a 3D space

#### Then plot 3D color map
##### Create meshgrid for 3D color map
The mesh can safely be created using hh0 data from the dataset measured at any value of magnetic field, since we've shown [that all hh0 data are the same within the linecut_f dataframe](#hh0_consistency)

##### Create the 2D array that contains the intensity data

##### Plot the hh0mesh and Hmesh data to see how irregular they are

##### Conclusion
The hh0 data looks pretty regular, however the magnetic field data is not
##### Update
That should not prevent from plotting in 3D

#### Use griddata to interpolate the intensity data over a regular array

#### Plot 3D surface
Note: Matplotlib does not do a very good job of plotting the 3D surface when the xlim is not adapted to the range of the data and when there is noise in the data: in our case, it looks like spectra are plotted individually rather than as a continuous surface. When zooming on each peak, the surface looks a little better. Perhaps look for a better 3D visualization tool.

#### Matplotlib example of 3D surface plotting

## Fit individual dataset above Hc

### Define fit functions

In [31]:
def plot_fit_result(x, y, fitresult):
    """
    Plot result of fit using an lmfit Model object
    """
    fig = plt.figure()
    plt.plot(x, y, 'bo')
    plt.plot(x, fitresult.init_fit, 'k--', label='initial fit')
    plt.plot(x, fitresult.best_fit, 'r-', label='best fit')
    plt.legend(loc='best')
    plt.show()
    

#### Attempt to an iteration callback function
Does not work as of 2020-04-14

In [35]:
def iter_func(params, iter_num, resid, x):
    print(iter_num)
    return True

### Fit example adapted from the lmfit documentation 
(section "Modeling Data and Curve Fitting)

In [30]:
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x-cen)**2 / wid)
gmodel = Model(gaussian)
print('parameter names: {}'.format(gmodel.param_names))
print('independent variables: {}'.format(gmodel.independent_vars))
gparams = gmodel.make_params(cen=5, amp=20, wid=1)
gparams

parameter names: ['amp', 'cen', 'wid']
independent variables: ['x']


name,value,initial value,min,max,vary
amp,20.0000000,None,-inf,inf,True
cen,5.00000000,None,-inf,inf,True
wid,1.00000000,None,-inf,inf,True


In [31]:
x = np.linspace(0,10,101)
noise = np.random.rand(101)-0.5
y = gaussian(x,amp=20,cen=5,wid=1)+noise
# fig = plt.figure()
# plt.plot(x, y)
# plt.show()
result = gmodel.fit(y, x=x, amp=5, cen=5, wid=1)
print(result.fit_report())
plot_fit_result(x, y, result)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 17
    # data points      = 101
    # variables        = 3
    chi-square         = 8.43330605
    reduced chi-square = 0.08605414
    Akaike info crit   = -244.776096
    Bayesian info crit = -236.930735
[[Variables]]
    amp:  19.9229410 +/- 0.10155032 (0.51%) (init = 5)
    cen:  4.99454353 +/- 0.00415645 (0.08%) (init = 5)
    wid:  0.99742555 +/- 0.01174109 (1.18%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, wid) = -0.577


### General parameters for our problem

In [34]:
peak_center = -8.0# center of unsplit peak to be studied in the following, in reciprocal space units
npeak_interval = .15# half of plot interval
Hc_0 = 0.51# value in Tesla units of the critical field at zero temperature
# in the absence of demagnetizing factor
# see data taken on needles of TmVO4-LS5200 in July 2017

### Single-peak fit with single pVIC function on single spectrum
measured above Hc, where the peaks are unsplit

#### Select data

In [37]:
dat_idx = 100
data_select = np.logical_and(nData.spectra[dat_idx].hh0 > peak_center - npeak_interval, 
                             nData.spectra[dat_idx].hh0 < peak_center + npeak_interval)
X = nData.spectra[dat_idx].hh0[data_select]# select data for first fit
Yfitdata = nData.spectra[dat_idx].Inorm[data_select]
X.head()

1940   -8.14875
1941   -8.14625
1942   -8.14375
1943   -8.14125
1944   -8.13875
Name: hh0, dtype: float64

#### Perform fit with Gaussian model as a first try

In [33]:
np.seterr(all='warn')# Set how floating-point errors are handled
# Setting warnings will help identify computation issues that may result in an unsuccessful fitting procedure
ngparams = gmodel.make_params(cen=peak_center, amp=4e6, wid=1e-3)
# Yeval = gmodel.eval(ngparams,x=X)# evaluate model using initial parameters
result = gmodel.fit(data=Yfitdata, params=ngparams, x=X)
print(result.fit_report())
plot_fit_result(X, Yfitdata, result)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 33
    # data points      = 120
    # variables        = 3
    chi-square         = 2.6305e+12
    reduced chi-square = 2.2483e+10
    Akaike info crit   = 2863.28327
    Bayesian info crit = 2871.64575
[[Variables]]
    amp:  4232624.39 +/- 61044.4590 (1.44%) (init = 4000000)
    cen: -7.99724868 +/- 2.1258e-04 (0.00%) (init = -8)
    wid:  3.2588e-04 +/- 1.0854e-05 (3.33%) (init = 0.001)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, wid) = -0.577


#### Single pVIC fit model

##### Make model from fit function

In [32]:
from ENS_peak_fit_pVIC_py.pseudoVoigtIkedaCarpenter import pVIC, xpVIC_residual, xpVIC_init_prm
pvic_model = Model(pVIC)# create Model object from the lmfit module
print(f'parameter names: {pvic_model.param_names}')
print(f'independent variables: {pvic_model.independent_vars}')

parameter names: ['A', 'alpha', 'beta', 'R', 'gamma', 'sigma', 'k', 'xp']
independent variables: ['x']


#### Fit parameters
##### Create fit parameters and specify their properties
including initial values, constraints, etc.

In [35]:
pvic_params = pvic_model.make_params(A=2e5, alpha=140, beta=1e-3, R=1e-3, 
                                     gamma=1e-3, sigma=6.6e-3, k=.05, xp=peak_center)
# Yfitdata = nData.spectra[0].Inorm
# pvic_params = pvic_model.guess(Yfitdata)# returns NotImplementedError
for k in pvic_params.keys():
    pvic_params[k].set(min=0, vary=True)
#     pvic_params[k].init_value = pvic_params[k].value
#     print(k)
pvic_params['xp'].set(min=-np.inf)
pvic_params['k'].vary = False
pvic_params

name,value,initial value,min,max,vary
A,200000.000,None,0.00000000,inf,True
alpha,140.000000,None,0.00000000,inf,True
beta,1.0000e-03,None,0.00000000,inf,True
R,1.0000e-03,None,0.00000000,inf,True
gamma,1.0000e-03,None,0.00000000,inf,True
sigma,0.00660000,None,0.00000000,inf,True
k,0.05000000,None,0.00000000,inf,False
xp,-8.00000000,None,-inf,inf,True


#### Evaluate fit function

In [38]:
np.seterr(all='warn')# Set how floating-point errors are handled
Yeval = pvic_model.eval(pvic_params,x=X)# evaluate model using initial parameters
fig = plt.figure()
plt.plot(X,Yeval)
plt.plot(X,Yfitdata)
plt.show()

#### Next steps as of 2020-04-14
* Play with values of beta to get best fit result; in particular, let beta vary (with max value of 800?)
* Is there a way to output the adjusted R**2 of the fit?

#### Narrowing down the number of free parameters

##### 7 free parameters
all but k

In [124]:
pvic_params['beta'].set(value=1e-10, vary=False)
pvic_params['R'].set(value=0, vary=False)
# pvic_params
result = pvic_model.fit(data=Yfitdata, params=pvic_params, x=X)
print(result.fit_report())
plot_fit_result(X, Yfitdata, result)

[[Model]]
    Model(pVIC)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 95
    # data points      = 120
    # variables        = 5
    chi-square         = 3.3204e+11
    reduced chi-square = 2.8873e+09
    Akaike info crit   = 2618.92465
    Bayesian info crit = 2632.86211
[[Variables]]
    A:      139800.679 +/- 1060.80779 (0.76%) (init = 200000)
    alpha:  130.275459 +/- 1.93231449 (1.48%) (init = 140)
    beta:   1e-10 (fixed)
    R:      0 (fixed)
    gamma:  2.2694e-07 +/- 1.8336e-04 (80796.03%) (init = 0.001)
    sigma:  0.00596552 +/- 2.3801e-04 (3.99%) (init = 0.0066)
    k:      0.05 (fixed)
    xp:    -8.01731207 +/- 2.3160e-04 (0.00%) (init = -8)
[[Correlations]] (unreported correlations are < 0.100)
    C(alpha, xp)    =  0.949
    C(A, gamma)     =  0.722
    C(gamma, sigma) = -0.480
    C(alpha, sigma) =  0.452
    C(sigma, xp)    =  0.444
    C(alpha, gamma) =  0.394
    C(gamma, xp)    =  0.355
    C(A, sigma)     = -0.346


The above yields too many correlations between parameters
The resulting plot also has an unphysical dip in the resulting fit function

##### 6 free parameters

In [66]:
pvic_params['beta'].set(value=1e-10, vary=False)
pvic_params['R'].set(value=0, vary=False)
result = pvic_model.fit(data=Yfitdata, params=pvic_params, x=X)
print(result.fit_report())
plot_fit_result(X, Yfitdata, result)
freeParams = [k for k in list(pvic_params.keys()) if pvic_params[k].vary==True]
plt.title(f"{len(freeParams)} free parameters: {freeParams}")

[[Model]]
    Model(pVIC)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 105
    # data points      = 120
    # variables        = 5
    chi-square         = 3.3204e+11
    reduced chi-square = 2.8873e+09
    Akaike info crit   = 2618.92398
    Bayesian info crit = 2632.86144
[[Variables]]
    A:      139800.970 +/- 832.210128 (0.60%) (init = 200000)
    alpha:  130.269301 +/- 1.82009419 (1.40%) (init = 140)
    beta:   1e-10 (fixed)
    R:      0 (fixed)
    gamma:  1.0779e-07 +/- 1.4438e-04 (133947.70%) (init = 0.001)
    sigma:  0.00596479 +/- 2.2440e-04 (3.76%) (init = 0.0066)
    k:      0.05 (fixed)
    x0:    -8.01731281 +/- 2.1916e-04 (0.00%) (init = -8)
[[Correlations]] (unreported correlations are < 0.100)
    C(alpha, x0)    =  0.942
    C(alpha, sigma) =  0.641
    C(sigma, x0)    =  0.631
    C(A, gamma)     =  0.472
    C(gamma, sigma) = -0.366
    C(A, x0)        = -0.227
    C(alpha, gamma) =  0.219
    C(A, alpha)     = -0.212
    C(A, sig

Text(0.5, 1.0, "5 free parameters: ['A', 'alpha', 'gamma', 'sigma', 'x0']")

## Batch fitting function


In [46]:
def initXpvicFit(fitObjStr, data, ref_params, h, fit_interval, data_range, 
                 nFunc=1, nFreeParams=None, fixParams={}, recompute=False):
    
    ref_params['xp'].value = -float(h)

    # Create key fot fitObj dictionary
    if nFreeParams is None:
        fitKeyBase = f"{h}_{nFunc}_{data_range[0]}"
    else:
        fitKeyBase = f"{h}_{nFunc}-{nFreeParams}_{data_range[0]}"

    if len(data_range)==1:
        fitKey = fitKeyBase
    else:
        fitKey = f"{fitKeyBase}-{data_range[-1]}" 

    # Create a dictionary that will contain xpvic_fit objects, if it doesn't exist yet
    if fitObjStr in globals() and type(eval(fitObjStr)) is dict:
        xFit = eval(fitObjStr)
    else:
        xFit = {}

    if fitKey not in xFit.keys() or recompute is True:
        # List of fit objects
        xFit[fitKey] = [xpvic_fit(data, ref_params, h, fit_interval, data_range, nFunc)]
        xFit[fitKey][0].initParams(fixParams=fixParams)

    if not hasattr(xFit[fitKey][0], 'result'):
        xFit[fitKey][0].performFit(with_weights=True)
    
    return xFit

## Batch fit data above Hc 
to determine the values of shared parameters

### Import fit function and make an lmfit model out of it

In [19]:
from ENS_peak_fit_pVIC_py.pseudoVoigtIkedaCarpenter import pVIC, xpVIC_residual, xpVIC_init_prm, xnpVIC_init_prm
pvic_model = Model(pVIC)# create Model object from the lmfit module
print(f'parameter names: {pvic_model.param_names}')
print(f'independent variables: {pvic_model.independent_vars}')

parameter names: ['A', 'alpha', 'beta', 'R', 'gamma', 'sigma', 'k', 'xp']
independent variables: ['x']


In [20]:
from ENS_peak_fit_pVIC_py.batch_pVIC_fitting import fixSharedParams
from ENS_peak_fit_pVIC_py.xpvic_fit_cls import xpvic_fit

### Batch fitting of single peak spectra with single pVIC function
above Hc

This will help fix the value of the parameters shared by all spectra, i.e. alpha, beta, R, gamma and sigma

#### Create fit parameters and specify their properties
including initial values, constraints, etc.
Note: when all pVIC parameters (except k) are free, the fit is very sensitive to initial values and can easily produce NaN numbers.
The following initial parameters appear to yield a result without raising any error:
A=2e5, alpha=140, beta=1e-3, R=1e-3, gamma=1e-3, sigma=6.6e-3, k=.05, xp=peak_center

In [21]:
pvic_params = pvic_model.make_params(A=2e5, alpha=140, beta=1e-3, R=1e-3, 
                                     gamma=1e-3, sigma=6.6e-3, k=.05, xp=-8.)
for k in pvic_params.keys():
    pvic_params[k].set(min=0, vary=True)
pvic_params['xp'].set(min=-np.inf)
pvic_params['k'].vary = False
pvic_params

name,value,initial value,min,max,vary
A,200000.000,None,0.00000000,inf,True
alpha,140.000000,None,0.00000000,inf,True
beta,1.0000e-03,None,0.00000000,inf,True
R,1.0000e-03,None,0.00000000,inf,True
gamma,1.0000e-03,None,0.00000000,inf,True
sigma,0.00660000,None,0.00000000,inf,True
k,0.05000000,None,0.00000000,inf,False
xp,-8.00000000,None,-inf,inf,True


#### Create xpvic_fit object and fit with all shared parameters free

In [26]:
fit_intervals = {6:[-.038, .15], 8:[-.1, .1], 10:[-.15, .15]}#
data_range = range(len(nData)-10, len(nData))

for h in [6,8,10]:
    xwFit = initXpvicFit('xwFit', nData, pvic_params, h, fit_intervals[h], data_range,
                         nFreeParams=2)

C:\Users\Pierre\Desktop\Postdoc\Software\Python\Python_scripts\Neutrons_pydas\ENS_peak_fit_pVIC_py\xpvic_fit_cls.py:144: RuntimeWarning: divide by zero encountered in true_divide
  self.weights = 1/(self.dY)
C:\Users\Pierre\Desktop\Postdoc\Software\Python\Python_scripts\Neutrons_pydas\ENS_peak_fit_pVIC_py\xpvic_fit_cls.py:150: UserWarning: Infinite values were encountered in 'weights', at positions                           []. They were reset to zero.
  {np.argwhere(self.weights==np.inf)}. They were reset to zero.")


#### Iteratively fix parameters shared between all datasets and recompute fit accordingly
The huge variability on R and beta seen when leaving those free implies that they play little role on the fit, so let's fix them both to zero.
Actually, beta=0 generates NaN data (not sure why) so let's fix it to 1e-10
For all other parameters, fix them to the best value obtained from the previous fit

In [47]:
def findBestSharedParams(fitObjStr, data, ref_params, h, fit_interval, data_range,
                         nFunc=1, recompute=False):

    # Create list of dictionaries containing the names of parameters to fix at each iteration
    fitKeyBase = f"{h}_{nFunc}_{data_range[0]}"
    if len(data_range)==1:
        fitKey = fitKeyBase
    else:
        fitKey = f"{fitKeyBase}-{data_range[-1]}" 

    freeSharedParams = ['alpha', 'beta', 'R', 'gamma', 'sigma']
#     ref_params['xp'].value = -float(h)

    # Initialize fit object if it doesn't exist yet
    xFit = initXpvicFit(fitObjStr, data, ref_params, h, fit_interval, data_range, recompute=recompute)

    # Loop to fix shared parameters and refit the data
    for idx in range(1,5):

        # When all shared parameters have been fixed, stop loop
        if len(freeSharedParams)==0:
            break

        # If necessary, create fit object and append it to fit dictionary
        if len(xFit[fitKey]) < idx+1:
            xFit[fitKey].append(xpvic_fit(data, ref_params, h, fit_intervals[h], data_range))
        
        freeSharedParams, fixedParams = fixSharedParams(freeSharedParams, xFit[fitKey][idx-1].result)

        if hasattr(xFit[fitKey][idx], 'result') and recompute is False :
            continue

        print(f"Iteration #{idx} of peak at ({h} {h} 0)")
        print(f"Free shared parameter: {freeSharedParams}")
        print(f"Fixed shared parameters: {fixedParams}\n")

        # create and initialize lmfit Parameters object using results of previous fit
        xFit[fitKey][idx].initParams(resultParams=xFit[fitKey][idx-1].result.params)
    #     xFit[fitKey][idx].initParams(resultParams=xFit['8_10'][2].result.params)

        for key, value in fixedParams.items():
            xFit[fitKey][idx].init_params[key].vary = False
            if value is not None:
                xFit[fitKey][idx].init_params[key].value = value

        # Compute the number of free shared parameters 
        xFit[fitKey][idx].freeSharedPrms = xFit[fitKey][idx-1].freeSharedPrms - len(fixedParams)

        # Perform fit
        xFit[fitKey][idx].performFit()

    return xFit

In [32]:
for h in [6,8,10]:
    xwFit = findBestSharedParams('xwFit', nData, pvic_params, h, fit_intervals[h], 
                                 data_range, recompute=False)

Iteration #1 of peak at (6 6 0)
Free shared parameter: ['alpha', 'gamma', 'sigma']
Fixed shared parameters: {'R': 1e-10, 'beta': 1e-10}

Iteration #2 of peak at (6 6 0)
Free shared parameter: []
Fixed shared parameters: {'alpha': None, 'gamma': None, 'sigma': None}

Iteration #1 of peak at (8 8 0)
Free shared parameter: ['alpha', 'gamma', 'sigma']
Fixed shared parameters: {'R': 1e-10, 'beta': 1e-10}

Iteration #2 of peak at (8 8 0)
Free shared parameter: []
Fixed shared parameters: {'alpha': None, 'gamma': None, 'sigma': None}

Iteration #1 of peak at (10 10 0)
Free shared parameter: ['alpha', 'gamma', 'sigma']
Fixed shared parameters: {'R': 1e-10, 'beta': 1e-10}

Iteration #2 of peak at (10 10 0)
Free shared parameter: []
Fixed shared parameters: {'alpha': None, 'gamma': None, 'sigma': None}



#### Results of weighted fit with 10 datasets on (8 8 0) peak 

In [25]:
fitKey = '8_1_92-101'
xwFit[fitKey][0].result

In [30]:
xwFit[fitKey][1].result

In [92]:
# report_fit(result)
xwFit[fitKey][2].result

#### Results of weighted fit with 10 datasets on (6 6 0) peak 

In [80]:
fitKey = '6_1_92-101'
xwFit[fitKey][0].result

In [81]:
xwFit[fitKey][1].result

In [82]:
xwFit[fitKey][2].result

#### Results of weighted fit with 10 datasets on (10 10 0) peak

In [31]:
# report_fit(result)
fitKey = '10_1_92-101'
xwFit[fitKey][0].result

In [71]:
# report_fit(result)
xwFit[fitKey][1].result

In [39]:
# report_fit(result)
xwFit[fitKey][2].result

#### Results of weighted fit with 20 datasets

In [70]:
# report_fit(result)
xwFit['8_20'][0].result

In [99]:
# report_fit(result)
xwFit['8_20'][1].result

In [100]:
# report_fit(result)
xwFit['8_20'][2].result

#### Fit with 2 independent free parameters per dataset and 5 free parameters shared between all datasets over 30 spectra

In [51]:
xwFit10b = [xpvic_fit(peak_center, nData, pvic_params)  for _ in range(numFitIterations)]

nSpec = 30
xwFit10b[0].data_range = range(len(nData)-nSpec, len(nData))

# create x and y 1D datasets for fit
# Xfitdata, Yfitdata = xyBatchFitNData(nData, data_range=spec_rng, data_select=data_select)
xwFit10b[0].makeData(data_select)

# create and initialize lmfit Parameters object
xwFit10b[0].initParams()

### Perform fit and store corresponding results in dictionary
xwFit10b[0].freeSharedPrms = 5

In [52]:
xwFit10b[0].performFit(weights=xwFit10b[0].weights)

In [38]:
# report_fit(result)
xwFit10b[0].result

In [36]:
# report_fit(result)
xFit10[0].result

In [37]:
# report_fit(result)
xwFit10[0].result

#### Plot fit result

In [154]:
idx = 2
plt_idx = 4
for h in [6, 8, 10]:
    plotKey = f'{h}_10'
    xwFit[plotKey][idx].plot_range = xwFit[plotKey][idx].data_range[-plt_idx-1:-plt_idx]
    xwFit[plotKey][idx].plotMultipleFits(title=f"Peak ({h} {h} 0)")

data_range index = 97;                   plot_range index = 97


#### Create Parameters object from result of fit of (8 8 0) peak

In [113]:
xfit8_params = cp.copy(pvic_params)
sharedParams = ['alpha', 'beta', 'R', 'gamma', 'sigma']
for prm_key in sharedParams:
    xfit8_params[prm_key] = xwFit['8_10'][2].result.params[prm_key]

<Parameter 'alpha', value=125.12346204356733 (fixed), bounds=[0:inf]>
<Parameter 'beta', value=1e-10 (fixed), bounds=[0:inf]>
<Parameter 'R', value=1e-10 (fixed), bounds=[0:inf]>
<Parameter 'gamma', value=0.000236804323940909 (fixed), bounds=[0:inf]>
<Parameter 'sigma', value=0.004901834233885571 (fixed), bounds=[0:inf]>


In [23]:
for h in range(6,11,2):
    nSpec = 10
    fitKey = f"{h}_{nSpec}"
    peak_pos = -float(h)
    xfit8_params['xp'].value = peak_pos
    xwFit[fitKey] = initXpvicFit(nData, nSpec, xfit8_params, peak_pos, fit_intervals[h])
    xwFit[fitKey][0].performFit(weights=xwFit[fitKey][0].weights)
#     xwFit[fitKey][0].performFit()

NameError: name 'xfit8_params' is not defined

## Fit data with 2 pVIC functions

#### Next steps as of 2020-05-08
* Email Jack and Jiajia to ask whether or not it makes sense to find different values of shared parameters for three different peaks?
* Fit with two pVIC functions at all fields, first with the amplitude ratio between two peaks free and then after fixing the ratio to the average at low fields H<.45T
* Extract relative splitting for all three values of h in both cases
* Fit splitting with a.sqrt(1-(H/Hc)^2) functional form, first with a different fit for each peak center value h=6,8,10, then with parameters a and hc shared between the three of them

Note: if code crashes when resuming on 2020-05-11, check the last modified line 
if fidx>0 and key=='A' in fixParams:
in xpvic_fit_cls.py

### Compute reference parameters

The npvicRefParams function allows to compute reference parameters from previous fit parameter results.
This is useful to initialize new reference parameters, especially after computing the optimal values of shared parameters.

In [36]:
def npvicRefParams(pvic_params, fit_params):

    ref_params = cp.copy(pvic_params)
    fit_params_keys = list(fit_params.keys())

    for index, key in enumerate(ref_params.keys()):
        if key in fit_params_keys[index]:
            ref_params[key] = cp.copy(fit_params[fit_params_keys[index]])
    
    return ref_params

In [41]:
n = 2
data_range = range(1)
plt_idx = data_range[0]

if 'ref_params' not in globals() or type(ref_params) is not dict:
    ref_params = {}

for h in [6, 8, 10]:
    if not hasattr(ref_params, f'{h}'):
        ref_params[h] = npvicRefParams(pvic_params, xwFit[f'{h}_1-4_92-101'][2].result.params)
#         fit2_params[h] = npvicInitParams(ref_params, plt_idx, n)

In [42]:
ref_params[8]

name,value,standard error,relative error,initial value,min,max,vary
A,146303.045,1482.31418,(1.01%),200000.0,0.00000000,inf,True
alpha,125.123375,0.00000000,(0.00%),125.12337489567476,0.00000000,inf,False
beta,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False
R,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False
gamma,2.3695e-04,0.00000000,(0.00%),0.00023694672897822677,0.00000000,inf,False
sigma,0.00490165,0.00000000,(0.00%),0.004901648479270548,0.00000000,inf,False
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp,-8.01798181,1.2594e-04,(0.00%),-8.0,-inf,inf,True


### Fit 2 peaks vs field

In [33]:
set_tup = (6, 8, 10)
fit_intervals = {6:[-.03, .15], 8:[-.1, .1], 10:[-.15, .15]}#

In [67]:
fix_params = {}
for h in set_tup:
    if 'fix_amp_ratio' in globals():
        fix_params[h] = {'A':fix_amp_ratio[h][0]}
    else:
        fix_params[h] = {}
        
fix_params

{6: {'A': 1.6409438030792185},
 8: {'A': 2.1097044913894143},
 10: {'A': 1.9427240060501394}}

In [78]:
# idx = 60
# fit_intervals[6] = [-.03, .15]
nfun = 2
nfree = 3

for idx in [0]:
    data_range = range(idx,idx+30)
    for h in [6]:
        xwFit = initXpvicFit('xwFit', nData, ref_params[h], h, fit_intervals[h], data_range, 
                             nFunc=nfun, nFreeParams=nfree, fixParams=fix_params[h], recompute=True)

In [79]:
idx = 0
fitKey = f"6_{nfun}-{nfree}_{idx}-{idx+29}"
# fitKey = f"6_{nfun}-{nfree}_{data_range[0]}-{data_range[-1]}"
xwFit[fitKey][0].result.params

name,value,standard error,relative error,initial value,min,max,vary,expression
A0_0,161586.156,979.850125,(0.61%),96206.04018959324,0.00000000,inf,True,
A0_1,98471.4746,597.125949,(0.61%),58628.47954272617,0.00000000,inf,False,A0_0/1.6409438030792185
alpha,206.679253,0.00000000,(0.00%),206.67925258024337,0.00000000,inf,False,
beta,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False,
R,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False,
gamma,7.2439e-04,0.00000000,(0.00%),0.0007243860167616134,0.00000000,inf,False,
sigma,0.00547680,0.00000000,(0.00%),0.005476803885552828,0.00000000,inf,False,
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False,
xp0_0,-5.99489735,8.9981e-05,(0.00%),-5.985,-inf,inf,True,
xp0_1,-6.02966216,1.3083e-04,(0.00%),-6.015,-inf,inf,True,


In [144]:
savepath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14\p6K\p6K_analysis\2020-05_p6K_da'
rcParams["figure.figsize"] = np.multiply([6.4, 4.8],1.5)
nfree = 4
for idx in (0, 30, 60):
    for h in set_tup:
        fitKey = f"{h}_{nfun}-{nfree}_{idx}-{idx+29}"
        xwFit[fitKey][0].plot_range = xwFit[fitKey][0].data_range[::len(data_range)//3]
        plot_spec = list(xwFit[fitKey][0].plot_range)
        title = f"TmVO$_4$ neutrons #{plot_spec}; {nfree} free params per curve"
        xwFit[fitKey][0].plotMultipleFits(title, plotSubFits=False)
        plt.xlim([-h-.1, -h+.1])
        savename = f"{str(date.today())}_TmVO4_p6K_ENS_2pVIC_weighted_{nfree}-free-params_fit_{h}-{h}-0_spec-num-{idx}-{idx+10}-{idx+20}.png"
#         plt.savefig('\\'.join([savepath,savename]), dpi=100)

rcParams["figure.figsize"] = np.multiply([6.4, 4.8],0.5)# default is [6.4, 4.8]

In [191]:
xwFit[fitKey][0].bestparams

array([[[ 1.90009241e+05,  9.54374390e+01,  1.00000000e-10,
          1.00000000e-10,  4.05789945e-04,  4.05638000e-03,
          5.00000000e-02, -9.99546535e+00],
        [ 9.82707081e+04,  9.54374390e+01,  1.00000000e-10,
          1.00000000e-10,  4.05789945e-04,  4.05638000e-03,
          5.00000000e-02, -1.00523084e+01]],

       [[ 1.93783312e+05,  9.54374390e+01,  1.00000000e-10,
          1.00000000e-10,  4.05789945e-04,  4.05638000e-03,
          5.00000000e-02, -9.99565160e+00],
        [ 9.97706863e+04,  9.54374390e+01,  1.00000000e-10,
          1.00000000e-10,  4.05789945e-04,  4.05638000e-03,
          5.00000000e-02, -1.00521997e+01]],

       [[ 1.95130162e+05,  9.54374390e+01,  1.00000000e-10,
          1.00000000e-10,  4.05789945e-04,  4.05638000e-03,
          5.00000000e-02, -9.99588404e+00],
        [ 1.00187504e+05,  9.54374390e+01,  1.00000000e-10,
          1.00000000e-10,  4.05789945e-04,  4.05638000e-03,
          5.00000000e-02, -1.00525156e+01]],

       [[ 

### Determine the average ratio of amplitudes vs magnetic field

In [52]:
def physQuantFromFit(fitObj, h, paramKey, operation='diff', nFunc=2, nFree=4, specPerRange=30, rangeNum=2):

    fitKeys = [f"{h}_{nFunc}-{nFree}_{idx}-{idx+specPerRange-1}" for idx in range(0,(rangeNum-1)*specPerRange+1,specPerRange)]
    total_num_spec = np.sum([fitObj[fitKey][0].num_spec for fitKey in fitKeys])
    physical_quantity = np.zeros((total_num_spec, 3))

    for fit_idx, fitKey in enumerate(fitKeys):
        for rng_idx, spec_idx in enumerate(fitObj[fitKey][0].data_range):
            idx = rng_idx + 30*fit_idx
            physical_quantity[idx,0] = fitObj[fitKey][0].data['H (T)'][spec_idx]
            params = {0:fitObj[fitKey][0].result.params[f'{paramKey}{spec_idx}_0'],
                      1:fitObj[fitKey][0].result.params[f'{paramKey}{spec_idx}_1']}
            if operation is 'diff':
                physical_quantity[idx,1] = np.abs(params[0].value - params[1].value) / h
                physical_quantity[idx,2] = np.abs(params[0].stderr + params[1].stderr) / h
            elif operation is 'div':
                physical_quantity[idx,1] = np.abs(params[0].value / params[1].value)
                physical_quantity[idx,2] = np.abs(params[0].stderr * params[1].value +
                                                  params[1].stderr * params[0].value) / params[1].value**2
            else:
                raise ValueError("Operation must be either 'diff' or 'div'.")
                
    return physical_quantity

In [61]:
if 'amplitude_ratio' not in globals():
    amplitude_ratio = {}

for h in set_tup:
    amplitude_ratio[h] = physQuantFromFit(xwFit, h, 'A', operation='div', nFree=4, 
                                          specPerRange=30, rangeNum=3)
#     fitKeys = [f"{h}_2_{idx}-{idx+29}" for idx in range(0,31,30)]

amplitude_ratio

{10: array([[0.00000000e+00, 1.93355420e+00, 6.21032978e-02],
        [4.99980800e-02, 1.94231549e+00, 7.91552398e-02],
        [9.99961600e-02, 1.94767229e+00, 7.90147923e-02],
        [1.50004960e-01, 1.93742266e+00, 8.07714399e-02],
        [2.00003040e-01, 1.93180899e+00, 8.03007857e-02],
        [2.50001120e-01, 1.99260107e+00, 8.38227902e-02],
        [2.99999200e-01, 1.88974909e+00, 7.87505982e-02],
        [3.20002720e-01, 1.91657310e+00, 7.97400557e-02],
        [3.39995520e-01, 1.94811131e+00, 8.30093669e-02],
        [3.59999040e-01, 1.95938069e+00, 8.39391363e-02],
        [3.80002560e-01, 1.90156357e+00, 8.02001079e-02],
        [3.99995360e-01, 1.98139282e+00, 8.56897990e-02],
        [4.19998880e-01, 2.00125513e+00, 8.58981188e-02],
        [4.40002400e-01, 1.91473571e+00, 8.28290863e-02],
        [4.59995200e-01, 1.96532943e+00, 8.52279761e-02],
        [4.79998720e-01, 2.01634466e+00, 8.94977163e-02],
        [5.00002240e-01, 2.11662692e+00, 9.46193354e-02],
        [5

In [62]:
linmod, linpar, linresult = {}, {}, {}

for h in set_tup:
    linmod[h] = LinearModel()
    xfit_sel = amplitude_ratio[h][:,0]<.45
    linpar[h] = linmod[h].make_params(slope=0, intercept=2)
    linpar[h]['slope'].vary = False
    linresult[h] = linmod[h].fit(amplitude_ratio[h][xfit_sel][:,1], linpar[h], 
                                  x=amplitude_ratio[h][xfit_sel][:,0])
    
    print(linresult[h].params)

Parameters([('slope', <Parameter 'slope', value=0 (fixed), bounds=[-inf:inf]>), ('intercept', <Parameter 'intercept', value=1.6409438030784353 +/- 0.00433, bounds=[-inf:inf]>)])
Parameters([('slope', <Parameter 'slope', value=0 (fixed), bounds=[-inf:inf]>), ('intercept', <Parameter 'intercept', value=2.1097044913896537 +/- 0.0122, bounds=[-inf:inf]>)])
Parameters([('slope', <Parameter 'slope', value=0 (fixed), bounds=[-inf:inf]>), ('intercept', <Parameter 'intercept', value=1.9427240060500142 +/- 0.00875, bounds=[-inf:inf]>)])


In [63]:
fig, ax = plt.subplots()
for h in [6,8,10]:
    x = amplitude_ratio[h][:,0]
    p = plt.errorbar(x, amplitude_ratio[h][:,1], amplitude_ratio[h][:,2],
                 marker='o', elinewidth=1, linewidth=0, label=f"({h}{h}0)")
    pcolor = p[-1][0].get_color()[0,:3]
    a = linresult[h].params['slope']
    b = linresult[h].params['intercept']
    plt.plot(x, a*x+b, color=pcolor)
plt.show()
plt.ylabel("Amplitude ratio")
plt.xlabel("$H$ (T)")
plt.ylim([0, 10])
plt.legend()

#### Attempt at a quadratic fit
Doesn't work and doesn't seem worth spending any more time on that

In [ ]:
def quad(x, a, b, c):
    return a*x**2 + b*x + c

def quad_init_param(params, dat_idx):
    

def quad_residual(params, xdata, ydata, spec_range, weights=None, nFunc=1):

    ndata, _ = ydata.shape # determine the number of datasets to fit
    resid = np.zeros(ydata.shape) # initialize array of residuals
    for spec_idx in range(ndata):
        
        model = np.sum([quad(xdata[spec_idx, :], 
                             *xnpVIC_init_prm(params, spec_range[spec_idx], fun_idx))
                        for fun_idx in range(nFunc)], axis=0)
        
        if weights is None:
            resid[spec_idx, :] = ydata[spec_idx, :] - model
        else:
            resid[spec_idx, :] = weights[spec_idx, :]*(ydata[spec_idx, :] - model)
            
    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()


In [269]:
quadmodel = quadmod[6] + quadmod[8] + quadmod[10]
quadparam = quadmodel.make

#### Constant amplitude ratio
Compute array of mean value of ratio of peak amplitudes for magnetic field < 0.45 T.
Check that this ratio is roughly constant at low fields by showing that the relative standard deviation is less than 10% (actually between 1% and 2%!)

In [65]:
fix_amp_ratio = {}
for h in set_tup:
    fix_amp_ratio[h] = np.array([np.mean(amplitude_ratio[h][amplitude_ratio[h][:,0]<.45][:,1]),
                                 np.std(amplitude_ratio[h][amplitude_ratio[h][:,0]<.45][:,1])])
    print(fix_amp_ratio[h][1]/fix_amp_ratio[h][0])

fix_amp_ratio

0.009504879403470948
0.020856830988086723
0.016242088399685533


{6: array([1.6409438 , 0.01559697]),
 8: array([2.10970449, 0.04400175]),
 10: array([1.94272401, 0.0315539 ])}

In [176]:
fix_amp_ratio[h].shape
np.zeros((2))#[:,0]#.shape[1]

array([0., 0.])

### Extract splitting vs field

In [98]:
if 'rel_splitting' not in globals():
    rel_splitting = {}

nfree = 3

for h in [6,8,10]:
    key = f"{h}-{nfree}"
    rel_splitting[key] = physQuantFromFit(xwFit, h, 'xp', nFunc=2, nFree=nfree, rangeNum=3)

rel_splitting

{'6-4': array([[0.00000000e+00, 5.79446283e-03, 3.57368458e-05],
        [4.99980800e-02, 5.75730631e-03, 4.53718070e-05],
        [9.99961600e-02, 5.70642067e-03, 4.58094231e-05],
        [1.50004960e-01, 5.68954730e-03, 4.60672852e-05],
        [2.00003040e-01, 5.59310420e-03, 4.62262694e-05],
        [2.50001120e-01, 5.48572324e-03, 4.62843040e-05],
        [2.99999200e-01, 5.33571776e-03, 4.65089900e-05],
        [3.20002720e-01, 5.30246892e-03, 4.63325331e-05],
        [3.39995520e-01, 5.24037522e-03, 4.64146649e-05],
        [3.59999040e-01, 5.15503983e-03, 4.63716813e-05],
        [3.80002560e-01, 5.08936091e-03, 4.66605068e-05],
        [3.99995360e-01, 4.97173316e-03, 4.70439207e-05],
        [4.19998880e-01, 4.86954436e-03, 4.72459123e-05],
        [4.40002400e-01, 4.81550981e-03, 4.76325218e-05],
        [4.59995200e-01, 4.69809409e-03, 4.83078105e-05],
        [4.79998720e-01, 4.60914550e-03, 4.89795175e-05],
        [5.00002240e-01, 4.49828278e-03, 4.94651811e-05],
       

In [110]:
plot_rel_split(rel_splitting, nfree, set_tup)

### Fit splitting vs magnetic field

In [87]:
def OP(h, a, hc):
    return a*np.sqrt(1-(h/hc)**2)

In [88]:
opmodel = Model(OP)
print('parameter names: {}'.format(opmodel.param_names))
print('independent variables: {}'.format(opmodel.independent_vars))

parameter names: ['a', 'hc']
independent variables: ['h']


In [89]:
opparams = opmodel.make_params(a=6e-3, hc=0.8)
for key in opparams.keys():
    opparams[key].min = 0

opparams

name,value,initial value,min,max,vary
a,0.00600000,None,0.00000000,inf,True
hc,0.80000000,None,0.00000000,inf,True


In [105]:
if 'rsplit_fitresult' not in globals():
    rsplit_fitresult = {}

maxfitfield = .7
for h in set_tup:
    key = f"{h}-{nfree}"
    rel_split_fit_range = rel_splitting[key][rel_splitting[key][:,0]<maxfitfield]
    rsplit_fitresult[key] = opmodel.fit(rel_split_fit_range[:,1], opparams, weights=1/rel_split_fit_range[:,2],
                                        h=rel_split_fit_range[:,0])

rsplit_fitresult[key].params

name,value,standard error,relative error,initial value,min,max,vary
a,0.00572925,9.3150e-06,(0.16%),0.006,0.00000000,inf,True
hc,0.78348853,0.00109482,(0.14%),0.8,0.00000000,inf,True


In [106]:
rsplit_fitresult[key].best_fit

array([0.00572925, 0.00571757, 0.0056824 , 0.00562326, 0.00553944,
       0.00542976, 0.00529262, 0.00522959, 0.00516169, 0.00508865,
       0.00501027, 0.00492635, 0.00483651, 0.00474046, 0.00463786,
       0.00452817, 0.0044109 , 0.00434924, 0.00428551, 0.00421949,
       0.00415112, 0.0040803 , 0.00404397, 0.0040069 , 0.00396922,
       0.00393076, 0.00389166, 0.00385182, 0.00381112, 0.00376972,
       0.0037274 , 0.00368433, 0.00364029, 0.00359544, 0.00354964,
       0.00350276, 0.00345497, 0.003406  , 0.00335603, 0.0033049 ,
       0.00325244, 0.00319882, 0.00314374, 0.00308736, 0.00302938,
       0.00296993, 0.00290882, 0.00284579, 0.00278099, 0.002714  ,
       0.00264496])

In [109]:
def plot_rel_split(relative_splitting, nfree, iterable, fit_result=None):
    fig, ax = plt.subplots()
    Hlin = np.concatenate([np.linspace(0,.7),.8-np.logspace(-1,-4)])
    for h in iterable:
        key = f"{h}-{nfree}"
        p = plt.errorbar(relative_splitting[key][:,0], relative_splitting[key][:,1], relative_splitting[key][:,2],
                     marker='o', elinewidth=1, linewidth=0, label=f"({h} {h} 0)")
        if fit_result is not None:
            a = fit_result[key].result.params['a']
            hc = fit_result[key].result.params['hc']
            pcolor = p[-1][0].get_color()[0,:3]
            plt.plot(Hlin, OP(Hlin, a, hc), color=pcolor)
            
    plt.show()
    plt.ylabel("Relative splitting")
    plt.xlabel("$H$ (T)")
    if nfree==3:
        plt.title("TmVO$_4$ distortion vs magnetic field, constrained amplitude ratio")
    elif nfree==4:
        plt.title("TmVO$_4$ distortion vs magnetic field, unconstrained amplitude ratio")
    plt.ylim([0, 6e-3])
    plt.legend()

In [97]:
fig, ax = plt.subplots()
Hlin = np.concatenate([np.linspace(0,.7),.8-np.logspace(-1,-4)])
for h in setlist:
    key = f"{h}-{nfree}"
    a = rsplit_fitresult[key].result.params['a']
    hc = rsplit_fitresult[key].result.params['hc']
#     rel_split_fit_range = rel_splitting[key][rel_splitting[key][:,0]<maxfitfield]
    p = plt.errorbar(rel_splitting[key][:,0], rel_splitting[key][:,1], rel_splitting[key][:,2],
                     marker='o', elinewidth=1, linewidth=0, label=f"({h} {h} 0)")
    pcolor = p[-1][0].get_color()[0,:3]
    plt.plot(Hlin, OP(Hlin, a, hc), color=pcolor)
plt.show()
plt.ylabel("Relative splitting")
plt.xlabel("$H$ (T)")
plt.ylim([0, 6e-3])
plt.legend()

In [135]:
rcParams["figure.figsize"] = np.multiply([6.4, 4.8],1.5)# default is [6.4, 4.8]
plot_rel_split(rel_splitting, 4, set_tup, fit_result=rsplit_fitresult)
# plot_rel_split(nfree, set_tup, rel_splitting, fit_result=rsplit_fitresult)